In [ ]:

import sys
sys.dont_write_bytecode = True
import pandas as pd
%load_ext autoreload
%autoreload 2


In [ ]:
df = full_data_loader('../dataset/itineraries_small.csv')

In [ ]:
df = data_prep(df, columns_path,distance_matrix)

In [ ]:
df_small = pd.read_csv("../dataset/df_ready_3000.csv")

In [ ]:
tracking_uri = 'http://0.0.0.0:5000'
exp_name = 'mlflow_logging'
train(df_small,tracking_uri, exp_name)

In [ ]:
import importlib

# Define the list of classes you want to import
sklearn_linear_model_imports = [
    'LinearRegression',
    'LogisticRegression',
    'Ridge',
    'Lasso'
]

# Function to dynamically import classes
def import_sklearn_classes(class_names):
    module = importlib.import_module('sklearn.linear_model')
    return {name: getattr(module, name) for name in class_names}

# Import the classes
imported_classes = import_sklearn_classes(sklearn_linear_model_imports)

# Now you can use the imported classes
for class_name, class_ref in imported_classes.items():
    print(f"Imported {class_name}: {class_ref}")

# Example usage
linear_regression = imported_classes['LinearRegression']()
print(f"Created LinearRegression model: {linear_regression}")

In [ ]:
linear_regression

In [91]:
import sys
sys.path.append('..')
sys.dont_write_bytecode = True
import pandas as pd
%load_ext autoreload
%autoreload 2
from mlops.tracking.mlflow import *



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
mlflow.set_tracking_uri('http://0.0.0.0:5000')
experiments = mlflow.search_experiments()

In [92]:
x = []
for exp in experiments:
    print(pick_top_run(exp,'r2_test'))

c1661f3cb5c6460ba435315b3033f757
6a0f11eaf46843a78011e26b29c90b6f
d2eeed76fc124a1baf2eca237c0e8fa6
eefd9e8a065d4351a0a73692714bce69


In [101]:
client.get_registered_model('lasso')

<RegisteredModel: aliases={}, creation_timestamp=1722487251129, description='', last_updated_timestamp=1722490129399, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1722490129399, current_stage='None', description='', last_updated_timestamp=1722490129399, name='lasso', run_id='eefd9e8a065d4351a0a73692714bce69', run_link='', source='mlflow-artifacts:/0/eefd9e8a065d4351a0a73692714bce69/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>], name='lasso', tags={}>

In [102]:
def get_model_info(model_name, metric_name='r2_test'):
    try:
        # Get the latest version of the model
        latest_version = client.get_latest_versions(model_name, stages=["None"])[0]
        
        # Get the run ID associated with this model version
        run_id = latest_version.run_id
        
        # Fetch the run data
        run = client.get_run(run_id)
        
        # Extract the desired metric
        metric_value = run.data.metrics.get(metric_name)
        
        return {
            "model_name": model_name,
            "version": latest_version.version,
            "run_id": run_id,
            f"{metric_name}": metric_value
        }
    except Exception as e:
        print(f"Error getting info for model {model_name}: {str(e)}")
        return None

In [104]:
get_model_info('lasso')['r2_test']

/tmp/ipykernel_522165/3311509924.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(model_name, stages=["None"])[0]


0.4513684562233631

In [95]:
run_id = "eefd9e8a065d4351a0a73692714bce69"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="lasso")

Registered model 'lasso' already exists. Creating a new version of this model...
2024/08/01 00:28:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lasso, version 4
Created version '4' of model 'lasso'.


<ModelVersion: aliases=[], creation_timestamp=1722490129399, current_stage='None', description='', last_updated_timestamp=1722490129399, name='lasso', run_id='eefd9e8a065d4351a0a73692714bce69', run_link='', source='mlflow-artifacts:/0/eefd9e8a065d4351a0a73692714bce69/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>

In [76]:
mlflow.search_registered_models().name

AttributeError: 'list' object has no attribute 'name'

In [75]:
exp.creation_time

1722320653302

In [42]:
client = MlflowClient()
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='0',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    
)

'eefd9e8a065d4351a0a73692714bce69'

In [46]:
for run in runs:
    print(run)
    print(f"run id: {run.info.run_id}")


<Run: data=<RunData: metrics={'r2_test': 0.4513684562233631,
 'training_mean_absolute_error': 102.36906912682669,
 'training_mean_squared_error': 19521.735150272423,
 'training_r2_score': 0.44606154087631233,
 'training_root_mean_squared_error': 139.72020308556822,
 'training_score': 0.44606154087631233}, params={'alpha': '1.0',
 'copy_X': 'True',
 'data size': '(800, 10)',
 'features': '["startingAirport", "destinationAirport", "isBasicEconomy", '
             '"isRefundable", "isNonStop", "departure_hour", "departure_day", '
             '"departure_month", "days_to_departure", "DISTANCE"]',
 'fit_intercept': 'True',
 'max_iter': '1000',
 'positive': 'False',
 'precompute': 'False',
 'random_state': 'None',
 'selection': 'cyclic',
 'tol': '0.0001',
 'warm_start': 'False'}, tags={'estimator_class': 'sklearn.linear_model._coordinate_descent.Lasso',
 'estimator_name': 'Lasso',
 'mlflow.log-model.history': '[{"run_id": "eefd9e8a065d4351a0a73692714bce69", '
                             '"

In [ ]:
exp_name ='mlflow_testing'

In [ ]:
from mlflow import MlflowClient
import mlflow

In [ ]:
from mlops.tracking import *

In [ ]:
mlflow.set_tracking_uri("http://0.0.0.0:5000")
mlflow.set_experiment('Lasson')


In [ ]:
mlflow.set_experiment(experiment_name=experiment_name)

In [ ]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
experiments = client.search_experiments()
for exp in experiments:
    print(exp.experiment_id, exp.name)
    print(mlflow.search_runs(exp.experiment_id))


In [ ]:
client = MlflowClient(tracking_uri=uri)

In [ ]:
runs = client.search_runs(experiment_id)
for run in runs:
    run
    print(run.info.run_id, run.info.status)

In [ ]:
run_id = client.search_runs(experiment_ids='0')[0].info.run_id

In [ ]:
runs

In [ ]:

experiment = mlflow.get_experiment_by_name(exp_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(exp_name)
else:
    experiment_id = experiment.experiment_id

# # Query all runs in the experiment
runs = mlflow.search_runs(experiment_ids=[experiment_id])a

# # Sort the runs by R2 test score and select the top three
# # Assuming the metric is named 'r2_test'
#top_runs = runs.sort_values("metrics.r2_test", ascending=False).head(3)

# # Define the names for the top 3 models
#top_model_names = ["Top_Model_1", "Top_Model_2", "Top_Model_3"]

# for index, (_, run) in enumerate(top_runs.iterrows()):
#     run_id = run.run_id
#     r2_score = run.metrics["r2_test"]
#     model_name = top_model_names[index]
    
#     # Load the model from the run
#     model_uri = f"runs:/{run_id}/model"
    
#     # Check if the model already exists
#     try:
#         existing_versions = client.get_latest_versions(model_name)
#     except mlflow.exceptions.RestException:
#         existing_versions = []
    
#     if existing_versions:
#         # If the model exists, create a new version
#         model_version = mlflow.register_model(model_uri, model_name)
#         print(f"Created new version of existing model: {model_name}, version: {model_version.version}, R2 score: {r2_score}")
#     else:
#         # If the model doesn't exist, register it
#         model_version = mlflow.register_model(model_uri, model_name)
#         print(f"Registered new model: {model_name}, version: {model_version.version}, R2 score: {r2_score}")
    
#     # Update the model description
#     client.update_registered_model(
#         name=model_name,
#         description=f"Model with R2 test score: {r2_score}"
#     )
    
#     # Add tags to the model version
#     client.set_model_version_tag(
#         name=model_name,
#         version=model_version.version,
#         key="r2_test_score",
#         value=str(r2_score)
#     )
    
#     # Transition the model to 'Production' stage
#     client.transition_model_version_stage(
#         name=model_name,
#         version=model_version.version,
#         stage="Production"
#     )
    
#     # Archive older versions
#     for old_version in existing_versions:
#         if old_version.version != model_version.version:
#             client.transition_model_version_stage(
#                 name=model_name,
#                 version=old_version.version,
#                 stage="Archived"
#             )

# print("Top 3 models have been updated and registered.")
